In [1]:
from operator import add
import pandas as pd
import itertools
from common import Data, Playoffs

In [2]:
df = pd.read_csv("data/2024.csv")
df["season"] = "2024"
teams = list(df["home"].unique())
Data.get_nwsl_standings(df)

,team,wins,draws,losses,goals_for,goals_against,goals_diff,points
1,ORL,15,6,0,38.0,12.0,26.0,51
2,WAS,14,2,5,42.0,24.0,18.0,44
3,NJY,13,4,4,26.0,17.0,9.0,43
4,KC,12,6,3,46.0,28.0,18.0,42
5,NC,10,2,9,28.0,23.0,5.0,32
6,CHI,9,2,10,27.0,28.0,-1.0,29
7,POR,8,4,9,31.0,30.0,1.0,28
8,LOU,6,7,8,30.0,29.0,1.0,25
9,BFC,8,1,12,25.0,33.0,-8.0,25
10,LA,6,4,11,25.0,35.0,-10.0,22


In [3]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("ORL", teams, g, p0)

4

In [4]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("WAS", teams, g, p0)

6

In [5]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_lowest_finish("KC", teams, g, p0)

7

In [6]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)
Playoffs.calculate_highest_finish("HOU", teams, g, p0)

5

In [7]:
lookforward = 7
start = min(df.index[df["home_score"].isna()])
outcomes = list([(0, 0), (1, 0), (0, 1)])

In [8]:
options = []
for i in range(lookforward):
    for outcome in outcomes:
        options.append((i, outcome))


# Summarize the scenarios into something meaningful
def format_result(i, outcome):
    match = df.iloc[start + i]
    return f"{match['home']} {outcome[0]} - {match['away']} {outcome[1]}"


def summarize(df_results):
    scenarios = df_results.copy()

    # Check if all values are true
    if sum(scenarios["clinched"]) == len(scenarios.index):
        print("Clinched independent of results")
        return

    # Check single game criteria
    for (g1, o1) in options:
        subset = scenarios[scenarios[g1] == o1]
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1))
            scenarios = scenarios[~(scenarios[g1] == o1)]

    # Check two game criteria
    for ((g1, o1), (g2, o2)) in itertools.combinations(options, 2):
        if g1 == g2:
            continue
        subset = scenarios[(scenarios[g1] == o1) & (scenarios[g2] == o2)]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1), "and", format_result(g2, o2))
            scenarios = scenarios[~((scenarios[g1] == o1) & (scenarios[g2] == o2))]

    # Check three game criteria
    for ((g1, o1), (g2, o2), (g3, o3)) in itertools.combinations(options, 3):
        if len(set([g1, g2, g3])) < 3:
            continue
        subset = scenarios[
            (scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
            )
            scenarios = scenarios[
                ~((scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3))
            ]

    # Check four game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4)) in itertools.combinations(options, 4):
        if len(set([g1, g2, g3, g4])) < 4:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                )
            ]

    # Check five game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4), (g5, o5)) in itertools.combinations(
        options, 5
    ):
        if len(set([g1, g2, g3, g4, g5])) < 5:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                )
            ]

    # Check six game criteria
    for (
        (g1, o1),
        (g2, o2),
        (g3, o3),
        (g4, o4),
        (g5, o5),
        (g6, o6),
    ) in itertools.combinations(options, 6):
        if len(set([g1, g2, g3, g4, g5, g6])) < 6:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
            & (scenarios[g6] == o6)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
                "and",
                format_result(g6, o6),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                    & (scenarios[g6] == o6)
                )
            ]

In [9]:
# Calculate all scenarios for the following week and whether a specific team
# has clinched a playoff spot
def evaluate_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_lowest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams)))
            )
            <= threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

In [10]:
# Calculate all scenarios for the following week and whether a specific team
# has been eliminated from the playoffs
def evaluate_elimination_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_highest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams)))
            )
            > threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

## Clinching playoffs

Already clinched: ORL, WAS, KC, NJY

In [12]:
summarize(evaluate_scenarios("NC", 8))

In [13]:
summarize(evaluate_scenarios("CHI", 8))

## Clinching hosting

Already clinched: ORL

In [14]:
summarize(evaluate_scenarios("WAS", 4))

Clinched with LA 0 - WAS 0 and CHI 1 - NC 0
Clinched with LA 0 - WAS 1 and CHI 0 - NC 0
Clinched with LA 0 - WAS 1 and CHI 1 - NC 0


In [15]:
summarize(evaluate_scenarios("KC", 4))

Clinched with KC 1 - NJY 0 and CHI 1 - NC 0


In [16]:
summarize(evaluate_scenarios("NJY", 4))

Clinched with KC 0 - NJY 1 and CHI 0 - NC 0
Clinched with KC 0 - NJY 1 and CHI 1 - NC 0


## Clinching shield

In [17]:
summarize(evaluate_scenarios("ORL", 1))

## Eliminated

In [18]:
summarize(evaluate_elimination_scenarios("HOU", 8))

In [19]:
summarize(evaluate_elimination_scenarios("UTA", 8))

Clinched with UTA 0 - LOU 1
